In [ ]:
#!pip install python-terrier

import numpy as np
import pandas as pd
import pyterrier as pt
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])



PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/M1_S2/RITAL/projet2
import json
import os

Mounted at /content/drive
/content/drive/MyDrive/M1_S2/RITAL/projet2


In [ ]:
!pip install python-terrier
import pandas as pd

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [5]:
#docs = json.load(open('WikiPassageQA/document_passages.json'))
# Loading json
with open("WikiPassageQA/document_passages.json",encoding="utf-8") as f:
    data = f.readlines()
    js_data = json.loads(data[0])


In [7]:
js_data['765']['0']

'The Appalachian Mountains , often called the Appalachians, are a system of mountains in eastern North America. The Appalachians first formed roughly 480 million years ago during the Ordovician Period. It once reached elevations similar to those of the Alps and the Rocky Mountains before naturally occurring erosion. The Appalachian chain is a barrier to east-west travel, as it forms a series of alternating ridgelines and valleys oriented in opposition to most roads running east or west. Definitions vary on the precise boundaries of the Appalachians. The United States Geological Survey defines the Appalachian Highlands physiographic division as consisting of thirteen provinces: the Atlantic Coast Uplands, Eastern Newfoundland Atlantic, Maritime Acadian Highlands, Maritime Plain, Notre Dame and Megantic Mountains, Western Newfoundland Mountains, Piedmont, Blue Ridge, Valley and Ridge, Saint Lawrence Valley, Appalachian Plateaus, New England province, and the Adirondack provinces. A commo

# Chargement des fichiers

### QA

On génére les fichiers :
- <code>relevant</code> : dataframe des passages pertinents pour chaque requete
- <code>train_queries</code>, <code>dev_queries</code>, <code>test_queries</code> : les dataframe de reqête

Les docno sont sous la forme {numéro doc},{numéro passage}

In [2]:
def string_to_list (s):
    """
    Converti un string de liste en une liste d'entier
    ex "21,22"->[21,22]
    """
    l=[]
    for i in s.split(',') :
        l.append(int(i))
    return l

In [ ]:
relevant = pd.DataFrame({'qid' : pd.Series(dtype='int'), 'docno' : pd.Series(dtype='str'), 'iteration' : pd.Series(dtype='float')})

def load_tsv (fname) :
    global relevant
    
    queries = pd.DataFrame({'qid' : pd.Series(dtype='int'), 'query' : pd.Series(dtype='str')})
    
    data = pd.read_csv(fname+'.tsv', sep='\t')
    for i in range(len(data)) :
        #ajout dans queries
        tupl=pd.DataFrame({'qid' : [data['QID'][i]],
                                      'query' : [data['Question'][i]]})
        queries=pd.concat([queries,tupl])
        
        #ajout dans relevant
        passages = string_to_list(data['RelevantPassages'][i])
        tupl=pd.DataFrame({'qid' : [data['QID'][i] for i in passages],
                           'docno' : [f"{data['DocumentID'][i]},{p}" for p in passages],
                           'iteration' : [1 for i in passages]})
        relevant=pd.concat([relevant,tupl])
    return queries



train_queries=load_tsv("data/train").reset_index().drop(columns=['index'])
dev_queries=load_tsv("data/dev").reset_index().drop(columns=['index'])
test_queries=load_tsv("data/test").reset_index().drop(columns=['index'])

relevant=relevant.reset_index().drop(columns=['index'])

In [ ]:
relevant

,qid,docno,iteration
0,956,"672,4",1.0
1,195,"359,1",1.0
2,557,"359,2",1.0
3,956,"285,4",1.0
4,2610,"579,14",1.0
...,...,...,...
6962,782,"393,2",1.0
6963,3738,"829,4",1.0
6964,844,"829,5",1.0
6965,1140,"364,1",1.0


In [ ]:
train_queries

,qid,query
0,3086,What is the role of conversionism in Evangelic...
1,195,How did the assault on the Bastille the first ...
2,557,What is the prehistory of Albania?
3,1508,What significance did Bulgaria have in the end...
4,956,What is the rationale of support of the Common...
...,...,...
3327,3700,How Do We Separate The Gray Wolf Or Grey Wolf ...
3328,1588,What is Greenland's self governing act?
3329,1971,How did conquering of Alexander the great effe...
3330,1153,What is Amnesty International?


### Documents

On charge le json dans un fichier pyterrier index

In [3]:
import json
pt_index_path = './passageQA'
if not os.path.exists(pt_index_path + "/data.properties") :
    # création de l'index
    indexer = pt.DFIndexer(pt_index_path, overwrite=True) 

    # chargement du json en dataframe
    with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
    data={"docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
          "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
    docs_df = pd.DataFrame(data)
    
    index_ref = indexer.index(docs_df["text"], docs_df["docno"])

else:
    # dans le cas où l'index existe déjà
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

In [4]:
docs_df

,docno,text
0,"344,1",Townspeople who lived in chartered towns were ...
1,"344,0",A municipal corporation is the legal term for ...
2,"344,3","This standard varies from state to state, acco..."
3,"344,2",All municipal corporations must also abide by ...
4,"344,5","Under the panchayati raj system, it interacts ..."
...,...,...
50607,"479,18",All dialects also divided in two main chronolo...
50608,"479,31","The official language in Moscow and Novgorod, ..."
50609,"479,30",It was soon followed by the adoption of Christ...
50610,"479,33",['knam va'kosk@ z@stU'tcit]\n\nThe political u...


In [ ]:
print(index.getCollectionStatistics().toString())

Number of documents: 50612
Number of terms: 103283
Number of postings: 3056988
Number of fields: 0
Number of tokens: 3906884
Field names: []
Positions:   false



## Combiner tout en 1 df

In [9]:
df_train = pd.read_csv("WikiPassageQA/train.tsv",sep='\t')
df_train

,QID,Question,DocumentID,DocumentName,RelevantPassages
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4
1,195,How did the assault on the Bastille the first ...,359,French_Revolution.html,"1,2"
2,557,What is the prehistory of Albania?,285,Albania.html,4
3,1508,What significance did Bulgaria have in the end...,579,Central_Powers.html,14
4,956,What is the rationale of support of the Common...,204,Common_Era.html,9
...,...,...,...,...,...
3327,3700,How Do We Separate The Gray Wolf Or Grey Wolf ...,174,Gray_wolf.html,0
3328,1588,What is Greenland's self governing act?,155,Greenland.html,4
3329,1971,How did conquering of Alexander the great effe...,641,Ancient_Egypt.html,"29,30"
3330,1153,What is Amnesty International?,740,Amnesty_International.html,"0,9"


In [8]:
df_test = pd.read_csv("WikiPassageQA/test.tsv",sep='\t')
df_test

,QID,Question,DocumentID,DocumentName,RelevantPassages
0,449,What is Iraq's role in political unstabilization?,543,Iraq.html,"42,43"
1,1140,What have been the warmest years of global war...,376,Global_warming.html,"7,8"
2,782,How were the Olympic games broadcasted?,699,2012_Summer_Olympics.html,"37,38,39,40"
3,1421,What was the election turnout for 2008?,131,"United_States_presidential_election,_2008.html",68
4,3738,How has secularism been misinterpreted in Amer...,441,Secularism.html,"10,11"
...,...,...,...,...,...
411,2522,What does the Church mean by Fallen Nature?,862,Eastern_Orthodox_Church.html,"24,25"
412,2851,What is the history of Eurasia?,393,Eurasia.html,"1,2"
413,3705,What are the nicknames of Missouri?,829,Missouri.html,"4,5"
414,799,How do endoglossic and exoglossic languages co...,364,Official_language.html,1


In [10]:
df_dev = pd.read_csv("WikiPassageQA/dev.tsv",sep='\t')
df_dev

,QID,Question,DocumentID,DocumentName,RelevantPassages
0,884,How did Mexico begin trying to get independenc...,61,Mexico.html,"22,23"
1,1362,What is the history of kindergarten?,190,Kindergarten.html,"0,1,2"
2,1998,How does a state get admitted to the union?,694,U.S._state.html,"16,17,18,19,20"
3,1909,What contributed to the mix of cultural influe...,297,Turkey.html,71
4,163,How did Greece expand during the Archaic period.,109,Ancient_Greece.html,"23,24"
...,...,...,...,...,...
412,343,Why were nearly all Blacks unable to vote in A...,822,Alabama.html,"17,18"
413,1875,How is the crime in Cleveland?,158,Cleveland.html,56
414,4079,What was the purpose of the one-party state?,78,Soviet_Union.html,13
415,1401,What distinguishes GDP from GNI?,424,Gross_domestic_product.html,13


In [11]:
df_query = pd.concat([df_train, df_test, df_dev])
df_query

,QID,Question,DocumentID,DocumentName,RelevantPassages
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4
1,195,How did the assault on the Bastille the first ...,359,French_Revolution.html,"1,2"
2,557,What is the prehistory of Albania?,285,Albania.html,4
3,1508,What significance did Bulgaria have in the end...,579,Central_Powers.html,14
4,956,What is the rationale of support of the Common...,204,Common_Era.html,9
...,...,...,...,...,...
412,343,Why were nearly all Blacks unable to vote in A...,822,Alabama.html,"17,18"
413,1875,How is the crime in Cleveland?,158,Cleveland.html,56
414,4079,What was the purpose of the one-party state?,78,Soviet_Union.html,13
415,1401,What distinguishes GDP from GNI?,424,Gross_domestic_product.html,13


In [13]:
import json
with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
data={"DocumentID" : [int(k0) for k0,v0 in data_row.items() for k1,v1 in v0.items()],
    "docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
      "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
docs_df = pd.DataFrame(data)

In [14]:
docs_df.head()

,DocumentID,docno,text
0,344,"344,1",Townspeople who lived in chartered towns were ...
1,344,"344,0",A municipal corporation is the legal term for ...
2,344,"344,3","This standard varies from state to state, acco..."
3,344,"344,2",All municipal corporations must also abide by ...
4,344,"344,5","Under the panchayati raj system, it interacts ..."


## merge avec passage

In [15]:
df_complete = df_query.merge(docs_df, on =["DocumentID"])
df_complete

,QID,Question,DocumentID,DocumentName,RelevantPassages,docno,text
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,42",They claimed major credit for the election of ...
1,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,48",Evangelicalism was a major force in the Anglic...
2,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,43",The rich and the poor remained traditional Cat...
3,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,49","The Evangelical Alliance, formed in 1846, was ..."
4,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,24","By 1737, Whitefield had become a national cele..."
...,...,...,...,...,...,...,...
244131,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,72",The budgetary rule is to spend no more than 4%...
244132,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,71",Norway is the fifth-largest oil exporter and t...
244133,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,70","In 2011, 28% of state revenues were generated ..."
244134,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,79",The two most prominent are the E6 going north-...


## creer qrel

In [26]:
# cree list relevant
relevant = []
for row in df_complete.iterrows():
  passage = row[1]['RelevantPassages'].split(",")
  doc_passage = row[1]['docno'].split(",")[1] # doc,pass -> [1]
  if doc_passage in passage:
    relevant.append(1)
  else:
    relevant.append(0)

In [27]:
df_complete["label"] = relevant
df_complete.head(30)

,QID,Question,DocumentID,DocumentName,RelevantPassages,docno,text,label
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,42",They claimed major credit for the election of ...,0
1,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,48",Evangelicalism was a major force in the Anglic...,0
2,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,43",The rich and the poor remained traditional Cat...,0
3,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,49","The Evangelical Alliance, formed in 1846, was ...",0
4,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,24","By 1737, Whitefield had become a national cele...",0
5,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,25","Pulling away from ritual and ceremony, the Gre...",0
6,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,26",Both the Evangelical and high church movements...,0
7,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,27",Evangelicals believed activism in government a...,0
8,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,20",At about the same time that Harris experienced...,0
9,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,21",He shared his voyage with a group of Moravian ...,0


In [29]:
qrel = df_complete[["QID", "docno","label"]]
qrel.head(30)

,QID,docno,label
0,3086,"672,42",0
1,3086,"672,48",0
2,3086,"672,43",0
3,3086,"672,49",0
4,3086,"672,24",0
5,3086,"672,25",0
6,3086,"672,26",0
7,3086,"672,27",0
8,3086,"672,20",0
9,3086,"672,21",0
